In [69]:
# coding=utf-8
import math
from scapy.all import *
import matplotlib.pyplot as plt
import numpy as np
import time
import pandas as pd
from scipy.stats import t
from math import sqrt
from math import pow

hosts = dict([('www.easternct.edu','149.152.33.32'), ('web.mit.edu','23.197.229.238')])

DEFAULT_RTT_VALUE = 0

ATTEMPS = 30

In [71]:
def modified_thompson(n):
    alpha = 0.025
    t_critical = t.ppf(1-alpha, n-2)
    return (t_critical*(n-1)) / (sqrt(n) * sqrt(n-2 + pow(t_critical,2)))  

In [72]:
def enviar_paquete(host, time_to_live):
    packet = IP(dst=host, ttl=time_to_live) / ICMP()
    res = sr(packet, timeout=1, verbose=0)
    return res[0][ICMP]

In [98]:
def route(df):
    return df[df['ip'] != 'empty']['ip'].unique()

In [73]:
def trace_route(hosts):
    
    hops = []

    for host in hosts:
        ttl_without_response = []
        ip = hosts[host]
        for i in range(0,ATTEMPS):
            tipo = 11
            ttl = 1
            last_rtt = 0
            while (tipo!=0 and ttl < 30):
                if (not ttl in ttl_without_response):
                    ti = time.time()
                    res1 = enviar_paquete(ip, ttl)
                    tf = time.time()
                    rtt = tf-ti
                    if (len(res1) > 0):
                        tipo = res1[0][1].type
                        src = res1[0][1].src
                        if (tipo > 0):
                            hops.append([host,ttl,i,src,max(0,rtt-last_rtt)])
                        else:
                            hops.append([host,ttl,i,src,rtt-last_rtt])
                        last_rtt = rtt
                    else:
                        hops.append([host,ttl,i,"empty",DEFAULT_RTT_VALUE])
                        ttl_without_response.append(ttl)
                ttl = ttl + 1

    hops_info = pd.DataFrame(hops,columns=['host','ttl','i','ip','rtt'])
    return hops_info

In [9]:
df = trace_route(dict([('www.uae.ma','159.253.148.195')]))

In [90]:
df_indexed = df.set_index(['host','ttl','i','ip'])
rtt = df_indexed.groupby(level=['host','ttl','ip']).mean()
descriptions = rtt.describe().T
mean = descriptions['mean']
std = descriptions['std']
zrtt = abs(rtt-mean)/std
n = len(zrtt)
thomson_value = modified_thompson(n)
print rtt
zrtt[zrtt['rtt'] > thomson_value]

                                     rtt
host       ttl ip                       
www.uae.ma 1   empty            0.000000
           2   empty            0.000000
           3   empty            0.000000
           4   empty            0.000000
           5   200.89.161.129   0.078922
               empty            0.000000
           6   empty            0.000000
           7   200.89.165.250  -0.000495
               empty            0.000000
           8   empty            0.000000
           9   195.22.219.3     0.066549
           10  195.22.219.3     0.002971
               empty            0.000000
           11  149.3.181.65     0.022690
               empty            0.000000
           12  129.250.2.227    0.102562
               empty            0.000000
           13  129.250.2.19     0.103181
           14  129.250.4.141    0.007698
           15  129.250.6.8     -0.007285
           16  81.25.207.146   -0.006672
               empty            0.000000
           17  5

rtt
host       ttl ip                     
www.uae.ma 12  129.250.2.227  2.580261
           13  129.250.2.19   2.598420

In [99]:
route(df)


array(['200.89.161.129', '200.89.165.250', '195.22.219.3', '149.3.181.65',
       '129.250.2.227', '129.250.2.19', '129.250.4.141', '129.250.6.8',
       '81.25.207.146', '50.97.18.211', '50.97.18.249', '159.253.158.131',
       '159.253.148.195'], dtype=object)